In [6]:
import re
import csv
import os

def parse_text_to_csv(input_file_path, output_file_path=None):
    # 如果未指定输出文件路径，则使用输入文件同名但扩展名为.csv
    if output_file_path is None:
        output_file_path = os.path.splitext(input_file_path)[0] + '.csv'

    # 读取输入文件内容
    with open(input_file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # 分割内容为多个记录（假设每个记录由空行分隔）
    records = content.strip().split('\n\n')

    # 定义CSV表头
    headers = [
        '股票名称', '总成交次数', '总成交金额(万元)', '总买入次数', '总买入金额(万元)',
        '总卖出次数', '总卖出金额(万元)',
        '早盘成交次数', '早盘成交金额(万元)', '早盘买入次数', '早盘买入金额(万元)',
        '早盘卖出次数', '早盘卖出金额(万元)',
        '中盘成交次数', '中盘成交金额(万元)', '中盘买入次数', '中盘买入金额(万元)',
        '中盘卖出次数', '中盘卖出金额(万元)',
        '尾盘成交次数', '尾盘成交金额(万元)', '尾盘买入次数', '尾盘买入金额(万元)',
        '尾盘卖出次数', '尾盘卖出金额(万元)',
        '详情'
    ]

    # 准备所有记录的数据
    all_data = []

    for record in records:
        # 跳过空记录
        if not record.strip():
            continue

        # 提取股票名称
        stock_name_match = re.match(r'(\S+)\s*，', record)
        if not stock_name_match:
            continue

        stock_name = stock_name_match.group(1)
        data = [stock_name]

        # 提取总体数据
        # 总成交次数和金额
        total_trades_match = re.search(r'成交(\d+)次', record)
        total_amount_match = re.search(r'成交总金额为(-?\d+)万元', record)

        # 买入次数和金额（注意：可能为负数）
        buy_times_match = re.search(r'买入(\d+)次', record)
        buy_amount_match = re.search(r'买入.*?金额(-?\d+)万元', record)

        # 卖出次数和金额（注意：可能为负数）
        sell_times_match = re.search(r'卖出(\d+)次', record)
        sell_amount_match = re.search(r'卖出.*?金额(-?\d+)万元', record)

        # 添加到数据列表
        data.append(total_trades_match.group(1) if total_trades_match else '0')
        data.append(total_amount_match.group(1) if total_amount_match else '0')
        data.append(buy_times_match.group(1) if buy_times_match else '0')
        data.append(buy_amount_match.group(1) if buy_amount_match else '0')
        data.append(sell_times_match.group(1) if sell_times_match else '0')
        data.append(sell_amount_match.group(1) if sell_amount_match else '0')

        # 提取各时间段数据
        time_periods = ['早盘', '中盘', '尾盘']
        for period in time_periods:
            # 使用更精确的正则表达式提取每个时间段的数据
            period_pattern = f'{period}：.*?成交(\\d+)次，成交金额(-?\\d+)万元；买入(\\d+)次金额(-?\\d+)万元；卖出(\\d+)次金额(-?\\d+)万元'
            match = re.search(period_pattern, record)
            if match:
                data.extend(match.groups())
            else:
                # 如果没有找到匹配，尝试更宽松的匹配
                loose_pattern = f'{period}：.*?成交(\\d+)次.*?成交金额(-?\\d+)万元.*?买入(\\d+)次.*?金额(-?\\d+)万元.*?卖出(\\d+)次.*?金额(-?\\d+)万元'
                loose_match = re.search(loose_pattern, record, re.DOTALL)
                if loose_match:
                    data.extend(loose_match.groups())
                else:
                    data.extend(['0'] * 6)

        # 提取详情
        detail_match = re.search(r'详情：(.*?)(?=\n|$)', record, re.DOTALL)
        detail = detail_match.group(1).strip() if detail_match else ''
        data.append(detail)

        all_data.append(data)

    # 写入CSV文件
    with open(output_file_path, 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(all_data)

    print(f"成功处理 {len(all_data)} 条记录，结果已保存到 {output_file_path}")
    return output_file_path

# 使用示例
if __name__ == "__main__":
    input_file = "C:\\Users\\16528\\PycharmProjects\\PythonProject\\save_file\\20250818\统计结果.txt"

    if os.path.exists(input_file):
        parse_text_to_csv(input_file)
    else:
        print("文件不存在，请检查路径是否正确")

成功处理 692 条记录，结果已保存到 C:\Users\16528\PycharmProjects\PythonProject\save_file\20250818\统计结果.csv
